<a href="https://colab.research.google.com/github/harishahamed26/YoutubeSA/blob/main/YoutubeSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube Sentimental Analysis

In [1]:
'''
Problem:
Scrapping the Youtube Video Comments and Analysing the sentiments using the LLM sentimental models in HuggingFace Library


1.ulian Lin
  Title:The STANFORD secret to learning 10x FASTER
  link:'https://www.youtube.com/watch?v=r8MLGFbOhYM'
  VideoId:'r8MLGFbOhYM'

2.Gravity Transformation - Fat Loss Experts
  Title:8 Reasons Your Shoulders Are NOT Growing (Science Based)
  link:'https://www.youtube.com/watch?v=c3wEEdD8tWc'
  VideoId:'c3wEEdD8tWc'

3.Jeremy Ethier
  Title: The #1 Workout That BLEW UP My Shoulders (3 Exercises)
  link:'https://www.youtube.com/watch?v=ufrFCjERMDc'
  VideoID:'ufrFCjERMDc'

4.Marcus Jones
  Title: I Studied 5,400 YT Shorts to CRACK the Algorithm
  link:'https://www.youtube.com/watch?v=ryZt5pc2O5s'
  VideoID:'ryZt5pc2O5s'

5.WWE
  Title: FULL SEGMENT: The Rock steps to Roman Reigns on Road To WrestleMania: SmackDown, Feb. 2, 2024
  link:'https://www.youtube.com/watch?v=58PpN0_VzQg'
  VideoID:'58PpN0_VzQg'

6.Olivia Rodrigo
  Title: Olivia Rodrigo - happier (Lyric Video)
  link:'https://www.youtube.com/watch?v=ZQFmRXgeR-s'
  VideoID:'ZQFmRXgeR-s'

'''

"\nProblem:\nScrapping the Youtube Video Comments and Analysing the sentiments using the LLM sentimental models in HuggingFace Library\n\n\n1.ulian Lin\n  Title:The STANFORD secret to learning 10x FASTER\n  link:'https://www.youtube.com/watch?v=r8MLGFbOhYM'\n  VideoId:'r8MLGFbOhYM'\n\n2.Gravity Transformation - Fat Loss Experts\n  Title:8 Reasons Your Shoulders Are NOT Growing (Science Based)\n  link:'https://www.youtube.com/watch?v=c3wEEdD8tWc'\n  VideoId:'c3wEEdD8tWc'\n\n3.Jeremy Ethier\n  Title: The #1 Workout That BLEW UP My Shoulders (3 Exercises)\n  link:'https://www.youtube.com/watch?v=ufrFCjERMDc'\n  VideoID:'ufrFCjERMDc'\n\n4.Marcus Jones\n  Title: I Studied 5,400 YT Shorts to CRACK the Algorithm\n  link:'https://www.youtube.com/watch?v=ryZt5pc2O5s'\n  VideoID:'ryZt5pc2O5s'\n\n5.WWE\n  Title: FULL SEGMENT: The Rock steps to Roman Reigns on Road To WrestleMania: SmackDown, Feb. 2, 2024\n  link:'https://www.youtube.com/watch?v=58PpN0_VzQg'\n  VideoID:'58PpN0_VzQg'\n\n6.Olivia Ro

In [ ]:
'''
AWS Task:
* create a notebook
* scrape the data from youtube
* figure out the preprocessing the data
* find how to store and retrieve in database
* itegrate the huggingface model
* get the result
* docker container
* deployment
* update in the github
'''

'\nAWS Task:\n* create a notebook\n* scrape the data from youtube\n* figure out the preprocessing the data\n* find how to store and retrieve in database\n* itegrate the huggingface model\n* get the result\n* docker container\n* deployment\n* update in the github\n'

In [2]:
!pip install boto3 google-api-python-client awscli emoji deep-translator tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 36.6 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [3]:
# Importing of Libraries and API keys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from googleapiclient.discovery import build
import datetime as dt

import boto3
from boto3.dynamodb.conditions import Key, Attr


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

youtubeAPI_key = '<api-key>'

## Step 1: Scrapping of Data From Youtube


In [12]:
# Defining a Function to scrape the youtube video comments

def YoutubeCommentScrapper(youtube, video, comments):
  request = youtube.commentThreads().list(
          part="snippet",
          maxResults=100,
          videoId= video
  )
  response = request.execute()

  for item in response['items']:
    channelId         = item['snippet']['topLevelComment']['snippet']['channelId']
    videoId           = item['snippet']['topLevelComment']['snippet']['videoId']
    textDisplay       = item['snippet']['topLevelComment']['snippet']['textDisplay']
    Prediction        = ' '

    comments.append([channelId,
                    videoId,
                    textDisplay,
                    Prediction])

  dataFrameComments = pd.DataFrame(comments,
                                   columns = ['channelId','videoId','textDisplay','Prediction'])
  return dataFrameComments

In [ ]:

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = youtubeAPI_key
youtube = build(api_service_name, api_version, developerKey = DEVELOPER_KEY)
videos = ['r8MLGFbOhYM','c3wEEdD8tWc','ufrFCjERMDc','ryZt5pc2O5s','58PpN0_VzQg','ZQFmRXgeR-s']
comments = []
for i in tqdm(videos):

  CommentsData  = YoutubeCommentScrapper(youtube,i,comments)

print(CommentsData)


## Step 2: Data Wrangling and Feature Selection


In [ ]:
'''CommentsData.to_csv('YoutubeSA_Data.CSV')
CommentsData = pd.read_csv('/content/YoutubeSA_Data.CSV')
CommentsData.drop('Unnamed: 0', axis= 1, inplace = True)
CommentsData.rename(columns = {'Unnamed: 0': 'Id'}, inplace=True)
CommentsData.rename(columns = {'P_N': 'Prediction'}, inplace=True)
'''

In [7]:
CommentsData.columns

Index(['channelId', 'videoId', 'textDisplay', 'Prediction'], dtype='object')

In [6]:
CommentsData.describe()

,channelId,videoId,textDisplay,Prediction
count,600,600,600,600
unique,6,6,594,1
top,UCL5PX6xQ0KBim6eQfDU74Qw,r8MLGFbOhYM,"Thank you so much, this past weeks i learned h...",
freq,100,100,2,600


In [7]:
CommentsData['textDisplay'].isna().sum()

0

In [5]:
# Converting all the strings to lower case

CommentsData['textDisplay'] = CommentsData['textDisplay'].str.lower()


In [6]:
# Removing Emoji

import emoji

CommentsData['textDisplay'] = CommentsData['textDisplay'].apply(lambda s: emoji.replace_emoji(s, ''))
CommentsData['textDisplay']

0      stanford neuroscientist andrew huberman explai...
1      thank you so much, this past weeks i learned h...
2                          bullshit trying to sell shit 
3      step 1 प्राणायाम<br>step 2 त्राथक<br>rest is  ...
4                                get to the point zzeeee
                             ...                        
595    estan si tienen un gran significado como la le...
596    my youngest kid she keep pplaying this and she...
597             i hope you happy but dont be happier !!!
598                                                     
599    who&#39;s going to be so stupid to break your ...
Name: textDisplay, Length: 600, dtype: object

In [7]:
# Removing Special characters

CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'<br>', '').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'</br>', '').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'<b>', '').str.strip()

CommentsData['textDisplay']

0      stanford neuroscientist andrew huberman explai...
1      thank you so much, this past weeks i learned h...
2                           bullshit trying to sell shit
3      step 1 प्राणायामstep 2 त्राथकrest is  especial...
4                                get to the point zzeeee
                             ...                        
595    estan si tienen un gran significado como la le...
596    my youngest kid she keep pplaying this and she...
597             i hope you happy but dont be happier !!!
598                                                     
599    who&#39;s going to be so stupid to break your ...
Name: textDisplay, Length: 600, dtype: object

In [8]:
# Removing the Hyper links

import warnings
warnings.filterwarnings("ignore")

url_regex1 = r'<a\s+href="(https:\/\/[^"]+)">(?:\d+:\d+)<\/a>'
url_regex2 =r'<a\s+href="([^"]+)">(\d{2}:\d{2})<\/a>'
url_regex3 =r'<a\s+href="([^"]+)">(\d+:\d+)<\/a>'


CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(url_regex1, '')
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(url_regex2,'')
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(url_regex3,'')


In [9]:
# HTML error changings
'''
Space ( ) is &#32
Exclamation mark (!) is &#33
Double quotation (“) is &#34
Number sign (#) is &#35
Percent sign (%) is &#37
Comma (,) is &#44
Hyphen (-) is &#45
Period (.) is &#46
Slash (/) is &#47
'''
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#32', ' ').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#33', '!').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#34', '"').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#35', '#').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#37', '%').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#39;', "'").str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#39', "'").str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#44', ',').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#45', '-').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#46', '.').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&#47', '/').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&quot;', '"').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'/', '').str.strip()
CommentsData['textDisplay'] = CommentsData['textDisplay'].str.replace(r'&amp;', '&').str.strip()
CommentsData['textDisplay']


0      stanford neuroscientist andrew huberman explai...
1      thank you so much, this past weeks i learned h...
2                           bullshit trying to sell shit
3      step 1 प्राणायामstep 2 त्राथकrest is  especial...
4                                get to the point zzeeee
                             ...                        
595    estan si tienen un gran significado como la le...
596    my youngest kid she keep pplaying this and she...
597             i hope you happy but dont be happier !!!
598                                                     
599     who's going to be so stupid to break your heart?
Name: textDisplay, Length: 600, dtype: object

In [10]:
CommentsData['textDisplay'].replace('', np.nan, inplace= True)

In [11]:
CommentsData.isna().sum()

channelId       0
videoId         0
textDisplay    17
Prediction      0
dtype: int64

In [12]:
CommentsData.dropna(inplace= True)

In [9]:

CommentsData.reset_index(inplace=True, drop=True)

In [10]:
CommentsData


,channelId,videoId,textDisplay,Prediction
0,UCL5PX6xQ0KBim6eQfDU74Qw,r8MLGFbOhYM,There&#39;s a contradiction there with the 10 ...,
1,UC0CRYvGlWGlsGxBNgvkUbAg,c3wEEdD8tWc,References<br><br>1. Studies show that full sq...,
2,UCERm5yFZ1SptUEU4wZ2vJvw,ufrFCjERMDc,"Hope you enjoyed this one, and shoutout to my ...",
3,UCR5k7bm0qgpP7-QX1uIx7cQ,ryZt5pc2O5s,Watch this video next to see me track down an ...,
4,UCJ5v_MCY6GNUBTO8-D3XoAg,58PpN0_VzQg,LOVE CODY&#39;S ENTRANCE,
5,UCxE5jEls-T0QtlTHT8lI1lw,ZQFmRXgeR-s,This song&gt;&gt;&gt;&gt;,


In [15]:
# Translate to English
from deep_translator import GoogleTranslator
from tqdm import tqdm

translator = GoogleTranslator(source='auto', target='en')

length = len(CommentsData['textDisplay'])

for i in tqdm(range(0,length)):
  CommentsData['textDisplay'][i] = translator.translate(CommentsData['textDisplay'][i])

CommentsData['textDisplay']

100%|██████████| 583/583 [02:14<00:00,  4.33it/s]


0      stanford neuroscientist andrew huberman explai...
1      thank you so much, this past weeks i learned h...
2                           bullshit trying to sell shit
3      step 1 Pranayama step 2 Trathakrest is especia...
4                                get to the point zzeeee
                             ...                        
578                              It's too beautiful here
579    They do have a great meaning like Taylor Swift...
580    my youngest kid she keep pplaying this and she...
581             i hope you happy but dont be happier !!!
582     who's going to be so stupid to break your heart?
Name: textDisplay, Length: 583, dtype: object

## Step 3: Import into DynamoDB using boto3

In [ ]:
!aws configure

In [ ]:
!aws configure set region <region-name> --profile <profile_name>

In [ ]:
!aws configure list

In [4]:
session = boto3.Session(region_name='<region-name>')
dynamo = session.resource('dynamodb')
Table_Name = 'YoutubeSA'

In [ ]:

# deleting the table

table = dynamo.Table(Table_Name)
table.delete()
table.wait_until_not_exists()

In [5]:
# Creating the table

Table_Name = 'YoutubeSA'
table_names = [table.name for table in dynamo.tables.all()]

if Table_Name in table_names:
    print('table', Table_Name, 'exists.')

    table = dynamo.Table(Table_Name)

else:
    # Creating the table
    print('The table', Table_Name, 'is not existed')
    print(f'Now creating a table {Table_Name}')

    table = dynamo.create_table(
    TableName=Table_Name,
    KeySchema=[
      {
          'AttributeName': 'Id',
          'KeyType': 'HASH'
      }

    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'N'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
    )
    print(f'The table has been created')



table YoutubeSA exists.


In [ ]:
# Inserting of CSV to Dynamo DB using Boto3
import datetime as dt;


for id, Values in tqdm(CommentsData.iterrows()):
  Creation_Timestamp = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  Update_Timestamp   = Creation_Timestamp
  response = table.put_item(
          Item = {
              'Id'                  : id,
              'channelId'           : Values['channelId'],
              'videoId'             : Values['videoId'],
              'textDisplay'         : Values['textDisplay'],
              'Prediction'          : Values['Prediction'],
              'Creation_Timestamp'  : Creation_Timestamp,
              'Update_Timestamp'    : Update_Timestamp
          }
          )

583it [01:12,  8.05it/s]


# Step 4: Model Generation and Prediction

In [6]:
from transformers import pipeline
Model_RobertaLarge = pipeline('zero-shot-classification', model='roberta-large-mnli')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [49]:
YoutubeSA_DB_Data = table.scan()

In [54]:
candidate_labels = ['positive','negative','neutral', 'contradiction','admiration', 'adoration',
                    'appreciation of beauty', 'amusement','anger', 'anxiety', 'awe',
                    'awkwardness', 'boredom', 'calmness', 'confusion', 'craving',
                    'disgust', 'empathic pain', 'entrancement', 'excitement', 'fear',
                    'horror', 'interest', 'joy', 'nostalgia', 'relief', 'sadness',
                    'satisfaction', 'surprise']

for items in tqdm(YoutubeSA_DB_Data['Items']):
  predict         = Model_RobertaLarge(items['textDisplay'], candidate_labels)
  Index           = pd.Series(predict['scores']).idxmax()
  PredictedValue  = predict['labels'][Index]

  # Update the result
  Update_Timestamp = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  table.update_item(
    Key                         = {'Id': items['Id']},
    UpdateExpression            = "SET Prediction = :value1, Update_Timestamp = :value2",
    ExpressionAttributeValues   ={
        ':value1': PredictedValue,
        ':value2': Update_Timestamp
        }
  )


100%|██████████| 201/201 [1:04:49<00:00, 19.35s/it]
